<a href="https://colab.research.google.com/github/SeokcheonMoon/study_data_analytics/blob/main/docs/CDA/CDA_RentalCarOfContractType_mixed_unNormal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 한글 폰트 설치
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 설치 후 colab의 경우 Runtime 이후 상단바 런타임의 Restart Session 을 클릭해야함
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용
import scipy
import scipy.stats as stats

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [ ]:
df_rental = pd.read_csv("RentalCarOfContractType.csv")
df_rental

,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,overdue_count,overdue,credit rating,bank,cancellation,age,Mileage
0,66758234,렌탈,Normal,서비스 방문,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,9.0,새마을금고,정상,43.0,1862.0
1,66755948,렌탈,Extension_Rental,서비스 방문,2019-10-20,60,카드이체,K1,102900,계약확정,0,없음,2.0,현대카드,정상,62.0,2532.0
2,66756657,렌탈,Normal,홈쇼핑/방송,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,8.0,우리은행,정상,60.0,2363.0
3,66423450,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,계약확정,0,없음,5.0,농협은행,정상,60.0,2449.0
4,66423204,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,해약확정,12,있음,8.0,농협은행,해약,51.0,1942.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51299,66579515,NaN,Promotion,R관리방판,2020-02-03,60,무통장,K3,96900,계약확정,0,없음,NaN,롯데카드,정상,NaN,NaN
51300,66799558,렌탈,Normal,영업방판,2020-02-03,60,카드이체,K1,96900,해약확정,0,없음,8.0,롯데카드,해약,39.0,1753.0
51301,66799197,렌탈,Promotion,홈쇼핑/방송,2020-02-03,39,무통장,K5,120900,해약확정,0,없음,1.0,NaN,해약,51.0,2217.0
51302,66792778,NaN,Normal,렌탈총판,2020-02-03,60,카드이체,K1,96900,계약확정,0,없음,2.0,신한카드,정상,64.0,2588.0


## 범주형 - 연속형 : 범주에 따른 관계성 확인

### 연속형이 비정규분포이며, 범주형 항목이 2개일때

#### 예제 가설

*   amount
*   channel(서비스 방문, 홈쇼핑) - 범주형 항목





#### 분석내용 : channel 중 서비스 방문과 홈쇼핑 선택에 따른 월 렌탈 비용 평균이 차이가 있는가?

- 귀무가설 : 비용평균의 차이가 없다.
- 대립가설 : 비용평균의 차이가 있다.


In [ ]:
df_rental_channel_two = df_rental["channel"].value_counts()[:2]
df_rental_channel_two

서비스 방문    15186
홈쇼핑/방송    12977
Name: channel, dtype: int64

In [ ]:
condition_service = "channel == '서비스 방문'"
condition_homeshop = "channel == '홈쇼핑/방송'"
condition_all = f"{condition_service} or {condition_homeshop}"
df_rental.query(condition_all)

,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,overdue_count,overdue,credit rating,bank,cancellation,age,Mileage
0,66758234,렌탈,Normal,서비스 방문,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,9.0,새마을금고,정상,43.0,1862.0
1,66755948,렌탈,Extension_Rental,서비스 방문,2019-10-20,60,카드이체,K1,102900,계약확정,0,없음,2.0,현대카드,정상,62.0,2532.0
2,66756657,렌탈,Normal,홈쇼핑/방송,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,8.0,우리은행,정상,60.0,2363.0
5,66165241,렌탈,Promotion,서비스 방문,2019-10-20,60,CMS,K3,90900,계약확정,0,없음,5.0,국민은행,정상,52.0,2001.0
6,66751017,렌탈,Package,홈쇼핑/방송,2019-10-20,60,카드이체,K1,98400,계약확정,0,없음,1.0,롯데카드,정상,59.0,2348.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51288,25731743,렌탈,Normal,서비스 방문,2020-02-03,60,CMS,K3,102900,계약확정,10,있음,NaN,국민은행,정상,NaN,NaN
51292,66792261,렌탈,Normal,홈쇼핑/방송,2020-02-03,60,카드이체,K1,96900,계약확정,0,없음,1.0,롯데카드,정상,46.0,1779.0
51296,66792778,렌탈,Normal,홈쇼핑/방송,2020-02-03,60,카드이체,K1,96900,계약확정,0,없음,2.0,롯데카드,정상,44.0,1947.0
51297,66799607,렌탈,Normal,홈쇼핑/방송,2020-02-03,60,CMS,K1,96900,해약확정,0,없음,8.0,신한은행,해약,70.0,2623.0


In [ ]:
conditions = "channel in ('서비스 방문', '홈쇼핑/방송')"
df_rental_channel_amount = df_rental.query(conditions)
df_rental_channel_amount[:2]

,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,overdue_count,overdue,credit rating,bank,cancellation,age,Mileage
0,66758234,렌탈,Normal,서비스 방문,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,9.0,새마을금고,정상,43.0,1862.0
1,66755948,렌탈,Extension_Rental,서비스 방문,2019-10-20,60,카드이체,K1,102900,계약확정,0,없음,2.0,현대카드,정상,62.0,2532.0


In [ ]:
df_rental.query(condition_service)["amount"].sum() # 서비스 방문 channel에 해당하는 amount의 총합

1496420760

In [ ]:
series_service_amount = df_rental.query(condition_service)["amount"]
series_service_amount

0         96900
1        102900
5         90900
8        102900
9        105900
          ...  
51231    105900
51234    105900
51247     96900
51282     96900
51288    102900
Name: amount, Length: 15186, dtype: int64

In [ ]:
series_homeshopping_amount = df_rental.query(condition_homeshop)["amount"]
series_homeshopping_amount

2         96900
6         98400
7         80400
10        81900
11        81900
          ...  
51272     96900
51292     96900
51296     96900
51297     96900
51301    120900
Name: amount, Length: 12977, dtype: int64

##### Wilcoxon Test

In [ ]:
stats.ranksums(series_service_amount,series_homeshopping_amount)

RanksumsResult(statistic=74.731078009603, pvalue=0.0)

#### 분석결과
- 통계 결과 : pvalue < 0.05
- 사실 결과 : 비용평균의 차이가 있다.


### 분석내용 : channel 중 서비스 방문, 홈쇼핑, 렌탈 총판 선택에 따른 월 렌탈 비용 평균이 차이가 있는가?
- 귀무가설 : 비용평균의 차이가 없다.
- 대립가설 : 비용평균의 차이가 있다.

In [ ]:
df_rental["channel"].value_counts()[:3].index

Index(['서비스 방문', '홈쇼핑/방송', '렌탈총판'], dtype='object')

In [ ]:
channel_list =  df_rental["channel"].value_counts()[:3].index.to_list()
channel_list

['서비스 방문', '홈쇼핑/방송', '렌탈총판']

In [ ]:
condition = "channel == '서비스 방문'"


In [ ]:
condition_all = f"{condition_service} or {condition_homeshop}"
df_rental.query(condition_service)["amount"][:5]

0     96900
1    102900
5     90900
8    102900
9    105900
Name: amount, dtype: int64

In [ ]:
series_list = list()
for channel in channel_list :
  condition = f"channel == '{channel}'"
  series_amount = df_rental.query(condition)["amount"]
  series_list.append(series_amount)
series_list[1].iloc[:5]

2     96900
6     98400
7     80400
10    81900
11    81900
Name: amount, dtype: int64

In [ ]:
stats.kruskal(series_list[0],series_list[1],series_list[2])

KruskalResult(statistic=8232.430933915304, pvalue=0.0)